# Notebook E-tivity 1 CE4021

Student name:

Student ID:

<hr style="border:2px solid gray"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.



## Task 1 : Symbolic calculation of the derivative

Description

Write 2 Python functions:

The first one to symbolically calculate the derivative of a polynomial with one variable. The input should be a polynomial and the output should also be a polynomial.

The second one to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar.

Test your code with a few salient polynomials (minimum of 3) for which you have calculated the derivative of these polynomials manually


# Polynomial

Polynomials are algebraic expressions that consist of variables and coefficients. An examples of a polynomial of a single variable x is x^2 − 4x + 7

4x^3 - 6x^2 + 9

(x + 4)^3 - 9X - 3

below are the few expressions which are not polynomial 

f(x) = 5x^4 - 2x^2 + 3 / x - divide by x, will be x^-1

f(x) = 4x^3 + sqrt(2) - square root

for this use case, we are going to use simplified expression 4x^3 - 6x^2 + 9 as complex functions required additional work to simplify it before applying derivatives

# Derivatives of polynomials

Derivatives of polynomials will primarily make use of power rule, more complex functions can be solved using chain rule and product rules are not in the scope of this e-tivity), so the power rule is

{d}/{dx} x^n = n x^{n - 1}

Example:

derivative of function 4x^3 - 6x^2 + 9 is 12x^2 - 12x

when we evaluate the polynomial at given point , let x=3 then the value of this function is 4*3^3 - 6*3^2 + 9 => 108 - 54 + 9 =63.

# Python data types
  There are many containerized data types available in python, we have coefficients and exponents in polynomial and 
both are associated with each other so Dictionary data type is best suited

Each term from polynomial will be represented in dictionary with power as key and coefficient as value

example 2*x^3 will be represented as 3: 2, 6*x will be 1: 6, and 7 will be 0: 7

we have many other options like list of tuples, list of dictionaries but those are useful when dealing with complex use cases
say for example if you have same power in more than one places we can use list of dict to represent them.

We can also represent using list like 5*x^3 + 7*X^2 + 6 as [5,7,3,6], here the order is important to get the power.
If you want to represent 2*x^1000 using list then we need to use additional zeros unnecessarily like [2,0,0,0,……….0] 


In [1]:
def derivative_of_polynomial(input_poly : dict) -> dict:     
     """This function is used to calculate the derivative_of_polynomial using power rule , 
     complex polynomials expressions(which uses chain rule and product rule) are not scope of this e-tivity 

    Parameters:
    argument1 (dict): input polynomial uses dictionary data type , can be represented below
    2*x^3 will be represented 3: 2, 6*x will be 1: 6, and 7 will be 0: 7

    Returns:
    dict : Returning derivative of polynomial which also a polynomial

   """
     der_of_poly={}
     for power, coeff in input_poly.items():
          if(power >= 1):
               der_of_poly[power-1] = coeff * power
     return der_of_poly

def evaluate_polynomial(input_poly : dict,x : int) -> int:     
     """This function is used to calculate the scaler value of polynomial

    Parameters:
    argument1 (dict): input polynomial uses dictionary data type , can be represented below
    2*x^3 will be represented 3: 2, 6*x will be 1: 6, and 7 will be 0: 7

    argument2 (int): given value of variable  x

    Returns:
    int : Returning numerical value of polynomial function for the given value of x

   """
     return sum(coeff * x**power for power, coeff in input_poly.items())

# Test the above functions using different inputs
print("****************Test Input 1*********************************************************************************")
input_polynomial_1= {100:3 , 2:1, 0:8}
print("Given polynomial is             : ",input_polynomial_1)
print("Derivative of polynomial is     : ", derivative_of_polynomial(input_polynomial_1))
print("Numeric value of polynomial is  ")
print("                     when x = 5 : ", evaluate_polynomial(input_polynomial_1,5))
print("                    when x = -4 : ", evaluate_polynomial(input_polynomial_1,-4))
print("                     when x = 0 : ", evaluate_polynomial(input_polynomial_1,0))    

print("****************Test Input 2*********************************************************************************")
input_polynomial_2= {10:-3 , 2:-1, 0:-8}
print("Given polynomial is             : ",input_polynomial_2)
print("Derivative of polynomial is     : ", derivative_of_polynomial(input_polynomial_2))
print("Numeric  value of polynomial is  ")
print("                     when x = 5 : ", evaluate_polynomial(input_polynomial_2,5))
print("                    when x = -4 : ", evaluate_polynomial(input_polynomial_2,-4))
print("                     when x = 0 : ", evaluate_polynomial(input_polynomial_2,0))  

print("****************Test Input 3*********************************************************************************")
input_polynomial_3= {1:7}
print("Given polynomial is             : ",input_polynomial_3)
print("Derivative of polynomial is     : ", derivative_of_polynomial(input_polynomial_3))
print("Numeric  value of polynomial is  ")
print("                     when x = 5 : ", evaluate_polynomial(input_polynomial_3,5))
print("                    when x = -4 : ", evaluate_polynomial(input_polynomial_3,-4))
print("                     when x = 0 : ", evaluate_polynomial(input_polynomial_3,0)) 

print("****************Test Input 4*********************************************************************************")
input_polynomial_4= {0:7}
print("Given polynomial is             : ",input_polynomial_4)
print("Derivative of polynomial is     : ", derivative_of_polynomial(input_polynomial_4))
print("Numeric  value of polynomial is  ")
print("                     when x = 5 : ", evaluate_polynomial(input_polynomial_4,5))
print("                    when x = -4 : ", evaluate_polynomial(input_polynomial_4,-4))
print("                     when x = 0 : ", evaluate_polynomial(input_polynomial_4,0)) 


****************Test Input 1*********************************************************************************
Given polynomial is             :  {100: 3, 2: 1, 0: 8}
Derivative of polynomial is     :  {99: 300, 1: 2}
Numeric value of polynomial is  
                     when x = 5 :  23665827156630354162351856958483586890196193053270690143108367919921908
                    when x = -4 :  4820814132776970826625886277023487807566608981348378505904152
                     when x = 0 :  8
****************Test Input 2*********************************************************************************
Given polynomial is             :  {10: -3, 2: -1, 0: -8}
Derivative of polynomial is     :  {9: -30, 1: -2}
Numeric  value of polynomial is  
                     when x = 5 :  -29296908
                    when x = -4 :  -3145752
                     when x = 0 :  -8
****************Test Input 3*********************************************************************************
Given polynomial is

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.